In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

from selenium.common.exceptions import TimeoutException

import json
import os
import sys

In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)

def teacher_norm(teacher):
    if not teacher.startswith('-'):
        return teacher
    tmp = teacher.split('\n')[0].split('- ')[1]
    return tmp[:-2]

def search_courses_by_date(date_str, courses):
    results = []
    
    date_str = date_str.strip()
    
    for item in courses:
        if date_str in item['date']:
            results.append(item)
    return results

def search_courses_by_teacher(teacher_str, courses):
    results = []
    
    teacher_str = teacher_str.strip()
    
    for item in courses:
        if teacher_str in item['teacher']:
            results.append(item)
    return results

def search_courses_by_name(name_str, courses):
    results = []
    
    name_str = name_str.strip()
    
    for item in courses:
        if name_str in item['name']:
            results.append(item)
    return results

def search_courses_by_time(time_str, courses):
    results = []
    
    time_str = time_str.strip()
    
    for item in courses:
        if time_str in item['time']:
            results.append(item)
    return results

    
def list_courses_to_be_booked(courses_to_be_booked):
    cur_date = ''
    if len(courses_to_be_booked) <= 0:
        return
    
    for course in courses_to_be_booked:  

        if cur_date != course['date']:
            if cur_date != '':
                print("-----------------------")
            print("Date: {}".format(course['date']))
            print("-----------------------\n")
            cur_date = course['date']
        print("Name: {}".format(course['name']))
        print("Teacher: {}".format(course['teacher']))
        print("Time: {}".format(course['time']))
        print("\n")
        

In [4]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [5]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [6]:
temp = []

# ---------------

with open(r'./class_schedules/12 April - 18 April 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)
# ---------------

# c = search_courses_by_date('08', course_schedule)
# c = search_courses_by_name('Warm', c)
# c = search_courses_by_teacher('Mei', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = search_courses_by_date('08', course_schedule)
# c = search_courses_by_name('Tone', c)
# c = search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = search_courses_by_date('08', course_schedule)
# c = search_courses_by_name('Warm', c)
# c = search_courses_by_teacher('Andy', c)
# c = search_courses_by_time('PM', c)
# temp.extend(c)

c = search_courses_by_date('18', course_schedule)
c = search_courses_by_name('Warm', c)
c = search_courses_by_teacher('Una', c)
# c = search_courses_by_time('PM', c)
temp.extend(c)

# c = search_courses_by_date('15', course_schedule)
# c = search_courses_by_name('Hatha', c)
# c = search_courses_by_teacher('Andy', c)
# # c = search_courses_by_time('PM', c)
# temp.extend(c)

# c = search_courses_by_date('09', course_schedule)
# c = search_courses_by_name('Warm', c)
# c = search_courses_by_teacher('Una', c)
# c = search_courses_by_time('PM', c)
# temp.extend(c)

courses_to_be_booked = temp

list_courses_to_be_booked(courses_to_be_booked)

Date: 18 Apr Sun
-----------------------

Name: Warm Stretch 溫和伸展
Teacher: Una
Time: 10:15 - 11:15 AM




In [7]:
master_counter = 1

In [8]:
def sign_in_once(driver, wait):
#     print("sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [9]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

In [10]:
csv_file_name = "./log_try_booking_a_class_that_was_full-- {}.csv".format(timestamp_string())

In [11]:
class_idx = 1
num_classes = len(courses_to_be_booked)

In [12]:
import sys

def reserve_class(driver, wait, course_to_be_booked):
    
    global master_counter
    global class_idx
    global num_classes

    stride =60 / 60 * 15 * num_classes

    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    teacher = course_to_be_booked['teacher']
    
#     reserve_class_log_dict = {}
    
#     reserve_class_log_dict['name'] = class_name
    driver.get(book_url)
    
    if click_book_this_class_now(driver, wait):
        
        sys.stdout.write("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',BOOKED,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        sys.stdout.write(log_str)
        
        with open(csv_file_name, "a") as write_file:
            write_file.write(log_str)
        num_classes = num_classes - 1
    else:
        
#         sys.stdout.write("master counter = {}, stride = {}, class_idx = {}".format(master_counter, stride, class_idx))

        log_str = "'{}','{}' by '{}',FULL,'{}'\n".format(master_counter, class_name, teacher, datetime.datetime.now())
        sys.stdout.write(log_str)
        
        if (master_counter - class_idx) % stride == 0:
            
            with open(csv_file_name, "a") as write_file:
                write_file.write(log_str)
        
        next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=60)
        scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
        master_counter = master_counter + 1
    
    if class_idx < num_classes:
        class_idx = class_idx + 1
    else:
        class_idx = 1


In [13]:
drivers = []
waits = []
scheduler = BackgroundScheduler()
start = time.time()

#CSV file table header
with open(csv_file_name, "w") as write_file:
            write_file.write("#,Class,Status,Time\n")
        
for idx, course_to_be_booked in enumerate(courses_to_be_booked):

    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    sign_in_once(driver, wait)
    next_book_datetime = datetime.datetime.now() + datetime.timedelta(seconds=15)
    scheduler.add_job(reserve_class, 'date', run_date= next_book_datetime, args=[driver, wait, course_to_be_booked])
    
# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_linux64.zip
[WDM] - Driver has been saved in cache [/home/huohsien/.wdm/drivers/chromedriver/linux64/89.0.4389.23]


'1','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:29:25.414644'
'2','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:30:46.090386'
'3','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:32:06.783488'
'4','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:33:27.483504'
'5','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:34:48.203918'
'6','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:36:08.866784'
'7','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:37:29.565794'
'8','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:38:50.289143'
'9','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:40:10.968175'
'10','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:41:31.669876'
'11','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:42:52.351590'
'12','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:44:13.041460'
'13','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:45:33.750060'
'14','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:46:54.416615'
'15','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 20:48:

'122','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:12:16.255900'
'123','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:13:36.968579'
'124','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:14:57.681639'
'125','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:16:18.380827'
'126','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:17:39.081453'
'127','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:18:59.798257'
'128','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:20:20.485590'
'129','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:21:41.195340'
'130','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:23:01.893361'
'131','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:24:22.593360'
'132','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:25:43.294637'
'133','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:27:03.992010'
'134','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:28:24.662035'
'135','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-16 23:29:45.406995'
'136','Warm Stretch 溫和伸展' by 'Una'

'241','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:52:19.134975'
'242','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:53:39.846615'
'243','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:55:00.523112'
'244','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:56:21.200319'
'245','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:57:41.902344'
'246','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 01:59:02.616458'
'247','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:00:23.295603'
'248','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:01:43.956251'
'249','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:03:04.669283'
'250','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:04:25.344380'
'251','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:05:46.014317'
'252','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:07:06.703395'
'253','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:08:27.427392'
'254','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 02:09:48.099665'
'255','Warm Stretch 溫和伸展' by 'Una'

'360','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:32:21.824515'
'361','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:33:42.534673'
'362','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:35:03.251378'
'363','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:36:23.948267'
'364','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:37:44.655998'
'365','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:39:05.361946'
'366','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:40:26.080602'
'367','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:41:46.777166'
'368','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:43:07.467221'
'369','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:44:28.167191'
'370','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:45:48.852870'
'371','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:47:09.561191'
'372','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:48:30.255518'
'373','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 04:49:50.967675'
'374','Warm Stretch 溫和伸展' by 'Una'

'479','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:12:24.143780'
'480','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:13:44.870974'
'481','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:15:05.571737'
'482','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:16:26.267909'
'483','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:17:46.962533'
'484','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:19:07.630888'
'485','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:20:28.339433'
'486','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:21:49.047604'
'487','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:23:09.741539'
'488','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:24:30.442544'
'489','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:25:51.104787'
'490','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:27:11.795044'
'491','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:28:32.502126'
'492','Warm Stretch 溫和伸展' by 'Una',FULL,'2021-04-17 07:29:53.202466'
'493','Warm Stretch 溫和伸展' by 'Una'